In [1]:
from pyspark.sql import SparkSession

# Stop any existing session
try:
    spark.stop()
except:
    pass

# Create NEW session with Kafka package
spark = SparkSession.builder \
    .appName("IoT Malware Detector") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .config("spark.sql.streaming.kafka.useDeprecatedOffsetFetching", "false") \
    .getOrCreate()

print("✅ Fresh Spark session with Kafka!")

✅ Fresh Spark session with Kafka!


In [7]:
df_kafka = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "big-data-final-project-kafka-1:29092") \
    .option("subscribe", "network-traffic") \
    .option("startingOffsets", "earliest") \
    .load()

print(f"✅ Connected to Kafka via IP!")
print(f"Total messages: {df_kafka.count()}")

✅ Connected to Kafka via IP!
Total messages: 156103


In [8]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

# Define schema based on our data structure
schema = StructType([
    StructField("ts", DoubleType()),
    StructField("id.orig_h", StringType()),
    StructField("id.orig_p", DoubleType()),
    StructField("id.resp_h", StringType()),
    StructField("id.resp_p", DoubleType()),
    StructField("proto", StringType()),
    StructField("duration", StringType()),
    StructField("orig_bytes", StringType()),
    StructField("resp_bytes", StringType()),
    StructField("conn_state", StringType()),
    StructField("label", StringType()),
    StructField("detailed-label", StringType())
])

# Parse JSON from Kafka value
df = df_kafka.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), schema).alias("data")) \
    .select("data.*")

# Show parsed data
df.show(5)
print(f"Total records: {df.count()}")

+-------------------+-----------+---------+--------------+---------+-----+--------+----------+----------+----------+---------+--------------------+
|                 ts|  id.orig_h|id.orig_p|     id.resp_h|id.resp_p|proto|duration|orig_bytes|resp_bytes|conn_state|    label|      detailed-label|
+-------------------+-----------+---------+--------------+---------+-----+--------+----------+----------+----------+---------+--------------------+
|  1.5267562618665E9|192.168.2.5|  38792.0|200.168.87.203|  59353.0|  tcp|2.998333|         0|         0|        S0|Malicious|PartOfAHorizontal...|
|1.526756268874876E9|192.168.2.5|  38792.0|200.168.87.203|  59353.0|  tcp|       -|         -|         -|        S0|Malicious|PartOfAHorizontal...|
|1.526756272877722E9|192.168.2.5|  38793.0|200.168.87.203|  59353.0|  tcp|2.997182|         0|         0|        S0|Malicious|PartOfAHorizontal...|
|1.526756279884959E9|192.168.2.5|  38793.0|200.168.87.203|  59353.0|  tcp|       -|         -|         -|       